In [ ]:
# Since My Kernel Keeps dying when running the optimize function, I was not able to get the results; 
# However, I tried to write some codes based on my understanding about the question to at least show my efforts before the ddl
# Will keep trying to see if I can improve further

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('use_inf_as_na', True)
import lightgbm as lgb
import pickle


import optuna
from optuna.trial import Trial
import optuna.integration.lightgbm as gbm

from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import train_test_split

from functools import partial

In [2]:
##P1 
##find the 10 most common features and use those at the optimal features.
with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/gb_opt_feat_1.pkl','rb') as f:
    optimal_features_1 = pickle.load(f)

with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/gb_opt_feat_2.pkl','rb') as f:
    optimal_features_2 = pickle.load(f)

with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/gb_opt_feat_3.pkl','rb') as f:
    optimal_features_3 = pickle.load(f)

opt_1 = [np.array(optimal_features_1[i]) for i in range(len(optimal_features_1))]
ordered_feat_1 = Counter(np.concatenate(opt_1,axis=0)).most_common()
most_common_feats_1 = [x[0] for x in ordered_feat_1[0:9]]

all_fea_1=[j for i in optimal_features_1 for j in i]
most_common_1=[x[0] for x in Counter(all_fea_1).most_common(10)]
all_fea_2=[j for i in optimal_features_2 for j in i]
most_common_2=[x[0] for x in Counter(all_fea_2).most_common(10)]
all_fea_3=[j for i in optimal_features_3 for j in i]
most_common_3=[x[0] for x in Counter(all_fea_3).most_common(10)]

In [9]:
#Construct two Gradient Boosting classifiers
raw_data = pd.read_pickle(r'/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/dataset.pkl')
data = raw_data[raw_data['market_cap'] > 1000.0]
data = data.copy()
data.fillna(0.0,inplace=True)

In [10]:
def f(x):
    if x > 0.01:
        return 1
    elif x < -0.01:
        return -1
    else:
        return 0
    
data['rel_performance'] = data['pred_rel_return'].apply(f)
data.reset_index(inplace=True,)
data.set_index('date',inplace=True)

In [11]:
df_1 = data.loc['2000-01-01':'2003-01-01']
df_valid = data.loc['2003-01-01':'2003-04-01']

train_1 = df_1.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1)

valid = df_valid.reset_index().drop(['ticker','date',
                                    'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1)

train_1_stock_returns = df_1['next_period_return']
valid_stock_returns = df_valid['next_period_return']

y_1 = df_1['rel_performance']
y_valid = df_valid['rel_performance']

y_1 = y_1.values
y_valid = y_valid.values

In [13]:
optuna.logging.set_verbosity(optuna.logging.FATAL)
import warnings
warnings.filterwarnings("ignore")

def objective(trial:Trial,train=None,labels=None,val=None,val_labels=None,val_rets=None):
    
    params = {
        "objective": "multi_class",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 5, 25),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',400,2400,step=400),
        'learning_rate': trial.suggest_float('learning_rate',0.01,0.2,step=0.01)
    }
    
    gb_clf = lgb.LGBMClassifier(**params)
    gbm = gb_clf.fit(train,labels)
    preds = gbm.predict(val)
    profit = (preds*val_rets).sum()
    return profit

In [14]:
study = optuna.create_study(direction="maximize")

In [ ]:
%%time
study.optimize(partial(objective,train=train_1,labels=y_1,val=valid,val_labels=y_valid,val_rets=valid_stock_returns), n_trials=100,n_jobs=1)

In [ ]:
study.best_params

In [17]:
gb_clf_1 = lgb.LGBMClassifier(**study.best_params)

In [29]:
my_params = {'reg_lambda': 5
                 ,'num_leaves': 10
                 ,'min_data_in_leaf': 1500
                 ,'learning_rate': 0.03}
gb_clf_2 = lgb.LGBMClassifier(**my_params)

In [30]:
# p/l

start_dates = [pd.to_datetime('2000-01-01') + pd.DateOffset(months = 3 * i) for i in range(62)]
end_dates = [d + pd.DateOffset(months = 36) for d in start_dates]

training_frames = [data.loc[d:d+pd.DateOffset(months = 36)] for d in start_dates]
valid_frames = [data.loc[d:d+pd.DateOffset(months = 3)] for d in end_dates]
test_frames =  [data.loc[d+pd.DateOffset(months = 3):d+pd.DateOffset(months = 6)] for d in end_dates]

training_data = [d.reset_index().drop
                                 (['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in training_frames]

valid_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in valid_frames]

test_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in test_frames]

training_labels = [d['rel_performance'].values for d in training_frames]
validation_labels = [d['rel_performance'].values for d in valid_frames]

In [ ]:
# most_common_1 and gb_clf_1

x_1 = [1]

for i in range(len(training_labels)):
        gb_clf_1.fit(training_data[i][most_common_1[:-1]],training_labels[i])
        pred_i = gb_clf_1.predict(test_data[i][most_common_feats_1])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        x_1.append(x_1[i] + (x_1[i]/num_positions) * profit_i)

In [ ]:
# most_common_1 and gb_clf_2

x_2 = [1]

for i in range(len(training_labels)):
        gb_clf_2.fit(training_data[i][most_common_1[:-1]],training_labels[i])
        pred_i = gb_clf_2.predict(test_data[i][most_common_feats_1])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        x_2.append(x_2[i] + (x_2[i]/num_positions) * profit_i)

In [ ]:
SPY = pd.read_pickle('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/SPY_cum_ret.pkl')
SPY = SPY.loc['2003-04-01':'2018-12-31']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['most_common_1 & gb_clf_1'] = x_1
SPY['most_common_1 & gb_clf_2'] = x_2
SPY.plot()

In [ ]:
#Most_common_2+gb_clf_1

x_3 = [1]

for i in range(len(training_labels)):
        gb_clf_1.fit(training_data[i][most_common_2[:-1]],training_labels[i])
        pred_i = gb_clf_1.predict(test_data[i][most_common_feats_1])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        x_3.append(x_1[i] + (x_1[i]/num_positions) * profit_i)

In [ ]:
#Most_common_2+gb_clf_2

x_4 = [1]

for i in range(len(training_labels)):
        gb_clf_1.fit(training_data[i][most_common_2[:-1]],training_labels[i])
        pred_i = gb_clf_2.predict(test_data[i][most_common_feats_1])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        x_4.append(x_1[i] + (x_1[i]/num_positions) * profit_i)

In [ ]:
SPY = pd.read_pickle('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/SPY_cum_ret.pkl')
SPY = SPY.loc['2003-04-01':'2018-12-31']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['most_common_2 & gb_clf_1'] = x_3
SPY['most_common_2 & gb_clf_2'] = x_4
SPY.plot()

In [ ]:
#Most_common_3+gb_clf_1

x_5 = [1]

for i in range(len(training_labels)):
        gb_clf_1.fit(training_data[i][most_common_3[:-1]],training_labels[i])
        pred_i = gb_clf_1.predict(test_data[i][most_common_feats_1])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        x_5.append(x_1[i] + (x_1[i]/num_positions) * profit_i)

In [ ]:
#Most_common_3+gb_clf_2

x_6 = [1]

for i in range(len(training_labels)):
        gb_clf_1.fit(training_data[i][most_common_3[:-1]],training_labels[i])
        pred_i = gb_clf_2.predict(test_data[i][most_common_feats_1])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        x_6.append(x_1[i] + (x_1[i]/num_positions) * profit_i)

In [ ]:
SPY = pd.read_pickle('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/SPY_cum_ret.pkl')
SPY = SPY.loc['2003-04-01':'2018-12-31']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['most_common_3 & gb_clf_1'] = x_5
SPY['most_common_3 & gb_clf_2'] = x_6
SPY.plot()

In [45]:
# P2

from sklearn.ensemble import RandomForestClassifier

def objective2(trial:Trial,train=None,labels=None,val=None,val_labels=None,val_rets=None):

    rf_n_estimators = trial.suggest_int('n_estimators', 12,42,step=8)
    rf_max_features = trial.suggest_categorical('max_features',['sqrt','log2'])
    rf_min_samples_leaf = trial.suggest_int('min_samples_leaf',1000,2000,step=1000)
    rf_max_depth = trial.suggest_int('max_depth',5,20)
    
    rf_clf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                    max_depth=rf_max_depth,
                                    min_samples_leaf=rf_min_samples_leaf,
                                    max_features=rf_max_features)
    rf_clf.fit(train,labels)
    preds = rf_clf.predict(val)
    profit = (preds * val_rets).sum()
 
    return profit

In [46]:
study2 = optuna.create_study(direction="maximize")

In [ ]:
%%time
study2.optimize(partial(objective2,train=train_1,labels=y_1,val=valid,val_labels=y_valid,val_rets=valid_stock_returns), n_trials=200,n_jobs=-1)

In [ ]:
print(study2.best_params)
rf_clf = RandomForestClassifier(**study2.best_params)

In [51]:
with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/opt_feat_1.pkl','rb') as f:
    rf_optimal_features_1 = pickle.load(f)

with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/opt_feat_2.pkl','rb') as f:
    rf_optimal_features_2 = pickle.load(f)

with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/opt_feat_3.pkl','rb') as f:
    rf_optimal_features_3 = pickle.load(f)

In [ ]:
%time
rf_x1 = [1]

for i in range(len(training_labels)):
        rf_clf.fit(training_data[i][rf_optimal_features_1[i]],training_labels[i])
        pred_i = rf_clf.predict(test_data[i][rf_optimal_features_1[i]])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        rf_x1.append(rf_x1[i] + (rf_x1[i]/num_positions) * profit_i)

In [53]:
%time
rf_x2 = [1]

for i in range(len(training_labels)):
        rf_clf.fit(training_data[i][rf_optimal_features_2[i]],training_labels[i])
        pred_i = rf_clf.predict(test_data[i][rf_optimal_features_2[i]])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        rf_x2.append(rf_x2[i] + (rf_x2[i]/num_positions) * profit_i)

In [54]:
%time
rf_x3 = [1]

for i in range(len(training_labels)):
        rf_clf.fit(training_data[i][rf_optimal_features_3[i]],training_labels[i])
        pred_i = rf_clf.predict(test_data[i][rf_optimal_features_3[i]])
        profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
        num_positions = len(pred_i)
        rf_x3.append(rf_x3[i] + (rf_x3[i]/num_positions) * profit_i)

In [ ]:
SPY = pd.read_pickle('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/SPY_cum_ret.pkl')
SPY = SPY.loc['2003-04-01':'2018-12-31']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['rf_opt_feat1'] = rf_x1
SPY['rf_opt_feat2'] = rf_x2
SPY['rf_opt_feat3'] = rf_x3
SPY.plot()

In [ ]:
average_rf = (SPY['rf_opt_feat1'] + SPY['rf_opt_feat2'] + SPY['rf_opt_feat3'])/3
SPY['average_rf'] = average_rf
SPY.plot()

In [57]:
# use most_common_1+gb_clf1 in my strategy

gb_strat=pd.DataFrame(x_1)

In [85]:
def performance(strat,SPY,strat_name):
    strat=pd.DataFrame(strat)
    strat_1_mean_ret = (strat - 1).diff().mean()
    strat_1_std = (strat - 1).diff().std()
    strat_1_sr = strat_1_mean_ret/strat_1_std
    print(f'{strat_name} Sharpe Ratio: ',strat_1_sr.values)

    strategy_ret = (SPY[strat_name] - 1).diff().values[1:]
    spy_ret = (SPY['spy_cum_ret'] - 1).diff().values[1:]
    beta = (np.cov(spy_ret,strategy_ret)/np.var(spy_ret))[1,0]
    print('Beta: ',beta)

    residual_ret = strategy_ret - beta * spy_ret
    IR = np.mean(residual_ret)/np.std(residual_ret)
    print('Information Ratio: ',IR)

    alpha = np.mean(residual_ret)
    print('Strategy alpha: ',alpha)

    i = np.argmax(np.maximum.accumulate(strat) - strat)
    j = np.argmax(strat[:i])
    plt.plot(strat)
    plt.plot([i, j], [strat[i], strat[j]], 'o', color='Red', markersize=10)
    print('Max drawdown:',np.abs(strat[j] - strat[i]))

In [62]:
SPY2 = pd.read_pickle('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 7/SPY_cum_ret.pkl')
SPY2 = SPY2.loc['2003-04-01':'2018-12-31']
SPY2 = SPY2.resample('Q').ffill()
SPY2['spy_cum_ret'] = (SPY2['spy_cum_ret'] - SPY2['spy_cum_ret'][0] + 1)

In [ ]:
# 30% gradient boosting strategy and 70% random forest strategy
strat_1 = 0.3 * gb_strat[0].values + 0.7 * average_rf.values
SPY2['my_strategy_1'] = strat_1
SPY2.plot()
performance(strat_1,SPY2,'my_strategy_1')

In [ ]:
# 50% gradient boosting strategy and short 50% random forest strategy

strat_2 = 0.5 * gb_strat[0].values - 0.5 * average_rf.values
SPY2['my_strategy_2'] = strat_2
SPY2.plot()
performance(strat_2,SPY2,'my_strategy_2')

In [ ]:
# 10% gradient boosting strategy and 90% random forest strategy

strat_3 = 0.1 * gb_strat[0].values + 0.9 * average_rf.values
SPY2['my_strategy_3'] = strat_3
SPY2.plot()
performance(strat_3,SPY2,'my_strategy_3')